In [1]:
import sys
import os
import numpy as np
import pickle
import plotly.graph_objects as go
from plotly.subplots import make_subplots

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from DEFAULTS import PARENT_PATH
import sim.tele_geo_ar as tg
import sim.pan_mod_ar as pm
import sim.fp_field_ar as fp

In [2]:
############# set the parametes #######################################
# propeties of the source transmitter
freq = 101.28 # frequency of signal source [GHz]
wavelength = (30.0 / freq) * 0.01 # [m]
wavelength = wavelength * 1e3 # [mm]
k = 2 * np.pi / wavelength
P_source = [0, -7200.0, 1e6] # unit [mm]
st_th_fwhp = 30.0/180.0*np.pi 

# scanning plane
Npts_x = 64
Npts_z = 64
scanrange_x = 64 # [mm]
scanrange_z = 64 # [mm]
P_rx_x = np.linspace(-scanrange_x/2.0, scanrange_x/2.0, Npts_x)
P_rx_y = 209.920654 # mm, position of focus point with rf source located 1km away
P_rx_z = np.linspace(-scanrange_z/2.0, scanrange_z/2.0, Npts_z)

# default telescope geometry
tele_geo = tg.TelescopeGeometry()

# arrays of pointing angles of rays
theta_a, theta_b, theta_N = -np.pi / 2 - 0.29, -np.pi / 2 + 0.29, 64
phi_a, phi_b, phi_N = np.pi / 2 - 0.29, np.pi / 2 + 0.29, 64
theta = np.linspace(theta_a, theta_b, theta_N)
phi = np.linspace(phi_a, phi_b, phi_N)

# create panel offsets randomly
save = 0 
error_rms = 0
adj_1_A = np.random.randn(1092) * error_rms
adj_2_A = np.random.randn(1092) * error_rms
panel_model2 = pm.panel_model_from_adjuster_offsets(
    2, adj_2_A, 1, save
)  # Panel Model on M2
panel_model1 = pm.panel_model_from_adjuster_offsets(
    1, adj_1_A, 1, save
)  # Panel Model on M1

# get parameters from telescope geometry
tg_th_1, tg_th2, tg_z_ap = tele_geo.th_1, tele_geo.th2, tele_geo.z_ap
tg_th_fwhp, tg_F_2 = tele_geo.th_fwhp, tele_geo.F_2
tg_rotc = np.array([tele_geo.x_rotc, tele_geo.y_rotc, tele_geo.z_rotc]) * 1e3 # [mm]

In [3]:
############# perform the simulation #######################################
# it could take several hours

field_fp = fp.focal_fields(
                           panel_model1, panel_model2, \
                           P_rx_x, P_rx_y, P_rx_z, P_source, \
                           tg_rotc, tg_th_1, tg_th2, tg_F_2, tg_z_ap, tg_th_fwhp, \
                           st_th_fwhp, theta, phi, k
                          )

In [4]:
############# save the simulation result #######################################
filename = "focal_field"
path = f"{PARENT_PATH}/data/tung"  # path where you save far-field simulations.
if not os.path.exists(path):
    os.makedirs(path)
with open(f"{path}/{filename}.pys", "wb") as file:
    pickle.dump(field_fp, file)
# with open(f"{path}/{filename}.pys", "rb") as file:
#     field_fp = pickle.load(file)

In [6]:
############# plot the field #######################################
import plotly.graph_objects as go


field_fp = field_fp / np.max(np.abs(field_fp)) # normalize the field
plot_1 = go.Heatmap(
                    x=P_rx_x, 
                    y=P_rx_z,
                    z = np.angle(field_fp),
                    colorscale='Twilight',
                    showscale=True, colorbar=dict(len=0.8, x=0.44),
                    showlegend = False,
                    # hoverinfo='name', 
                    name="phase"
                    )
plot_2 = go.Heatmap( 
                    x=P_rx_x, 
                    y=P_rx_z,
                    z = 20*np.log10(np.abs(field_fp)),
                    # z= np.abs(field_fp*field_fp),
                    colorscale='Magma',
                    showscale=True, 
                    colorbar=dict(len=0.8, x=1), 
                    showlegend = False,
                    # hoverinfo='name', 
                    name="power"
                    )
layout = go.Layout(title='Beam on focal plane', autosize=True,
                width=1000, height=500, 
                margin=dict(l=50, r=50, b=65, t=90),
                xaxis1 = dict(title="x [mm]"),
                yaxis1 = dict(scaleanchor = 'x', title="y [mm]"),
                xaxis2 = dict(scaleanchor = "x", title="x [mm]"),
                yaxis2 = dict(scaleanchor = "y"),
                )

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, shared_xaxes=True, subplot_titles=["Phase [rad]", "Intensity [dB]"])
fig.add_trace(plot_1, row=1, col=1)
fig.add_trace(plot_2, row=1, col=2)
fig.update_layout(layout)
fig.show()